In [44]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import random
import time
import itertools
import math
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [45]:
LENGTH=100
WIDE=50

TIME_LAYOUT1 = pd.DataFrame({
    'L/U':[0,12,10,8,6],
    'M1':[6,0,6,8,10],
    'M2':[8,6,0,6,8],
    'M3':[10,8,6,0,6],
    'M4':[12,10,8,6,0]},
    index = ['L/U','M1','M2','M3','M4'])

TIME_LAYOUT2 = pd.DataFrame({
    'L/U':[0,6,8,6,4],
    'M1':[4,0,12,10,8],
    'M2':[6,2,0,12,10],
    'M3':[8,4,2,0,12],
    'M4':[6,2,4,2,0]},
    index = ['L/U','M1','M2','M3','M4'])

TIME_LAYOUT3 = pd.DataFrame({
    'L/U':[0,12,10,4,2],
    'M1':[2,0,12,6,4],
    'M2':[4,2,0,8,6],
    'M3':[10,8,6,0,12],
    'M4':[12,10,8,2,0]},
    index = ['L/U','M1','M2','M3','M4'])

TIME_LAYOUT4 = pd.DataFrame({
    'L/U':[0,18,20,12,14],
    'M1':[4,0,14,8,14],
    'M2':[8,4,0,6,12],
    'M3':[10,6,8,0,6],
    'M4':[14,10,6,6,0]},
                            
    index = ['L/U','M1','M2','M3','M4'])

GANTT_CHART = pd.DataFrame({
    'M1':np.zeros(LENGTH),
    'M2':np.zeros(LENGTH),
    'M3':np.zeros(LENGTH),
    'M4':np.zeros(LENGTH)},
    index = [np.arange(1,LENGTH+1)],dtype=int)
#print(gantt_chart)

OPERATION = pd.DataFrame({
    'job':np.zeros(WIDE),
    'process':np.zeros(WIDE)},
    index = [np.arange(1,WIDE+1)],dtype=int)

state_M=np.zeros(4,dtype=int)
state_J1=np.zeros(5,dtype=int)
state_J2=np.zeros(6,dtype=int)
state_J3=np.zeros(6,dtype=int)
state_J4=np.zeros(5,dtype=int)
state_J5=np.zeros(5,dtype=int)
state_J6=np.zeros(6,dtype=int)
state_J7=np.zeros(8,dtype=int)
state_J8=np.zeros(6,dtype=int)
state_J9=np.zeros(5,dtype=int)
state_J10=np.zeros(6,dtype=int)


In [46]:
#JOBSET1

best_t=9999

MS_J1=np.array([[1,2,4],
               [1,3,2],
               [3,4,1],
               [4,2,-1],
               [3,1,-1]])#-1は作業なしと表示される

T_J1=np.array([[8,16,12],
               [20,10,18],
               [12,8,15],
               [14,18,-1],
               [10,15,-1]])#-1は作業なしと表示される

list_arrange_J1 = list(itertools.permutations(np.arange(0,5)))
list_time_J1=np.zeros(math.factorial(5))
gene=0

for a in list_arrange_J1:
    #初期化
    gantt_chart_J1 = pd.DataFrame({
        'M1':np.zeros(LENGTH),
        'M2':np.zeros(LENGTH),
        'M3':np.zeros(LENGTH),
        'M4':np.zeros(LENGTH)},
        index = [np.arange(1,LENGTH+1)],dtype=int)
    
    operation_J1=pd.DataFrame({
        'job':np.zeros(WIDE),
        'process':np.zeros(WIDE)},
        index = [np.arange(1,WIDE+1)],dtype=int)
    
    state_M=np.zeros(4,dtype=int)
    state_J1=np.zeros(5,dtype=int)
    check_J1=np.array([[0,0,0],
                       [0,0,0],
                       [0,0,0],
                       [0,0,-1],
                       [0,0,-1]]) #ジョブの確認表（0:まだしていない、1：もうした、-1：作業なし）
    o=1#operation迭代
    
    for t in np.arange(1,LENGTH+1):#時系列
        if np.any(state_M==0) and np.any(state_J1==0):#空いている機械と空いているジョブはあるかどうかを検査
            #for j in np.arange(0,3)#ジョブ1からジョブ５までループ
            for j in a:
                if state_J1[j]==0:#選んだジョブｊは空いているかどうかを検査
                    for i in np.arange(0,3):#選んだジョブｊの中のプロセスｊをループ
                        if check_J1[j,i]==0 and state_J1[j]==0:#選んだプロセス(i,j)はしたかどうかを検査、また循環の中でジョブｊは空いているかどうかを確認
                            if i==0 or (i>0 and check_J1[j,i-1]==1):#選んだ空いているプロセス(i,j)は手前のプロセスや前のプロセスはもうしたの場合
                                m=MS_J1[j,i]-1#プロセス(i,j)の作業機械番号を代入
                                if state_M[m]==0:#プロセス(i,j)の作業機械は空いているかどうかを確認
                                    state_M[m]=T_J1[j,i]#作業時間を機械状態に代入
                                    state_J1[j]=T_J1[j,i]#作業時間をジョブ状態に代入
                                    check_J1[j,i]=1#プロセス(i,j)にチャックを付く
                                    gantt_chart_J1.iloc[t-1:t-1+T_J1[j,i],m]=j+1#作業時間とジョブ番号を代入
                                    operation_J1.loc[o,'job']=j+1
                                    operation_J1.loc[o,'process']=i+1
                                    o+=1
        state_M[state_M !=0] -= 1#カウントダウン
        state_J1[state_J1 !=0] -= 1
        if np.all(check_J1!=0):#チェック表をチェック、作業全部済んだら、プールから出る
            break

    for t in np.arange(1,LENGTH+1):
        if np.all(gantt_chart_J1.iloc[t-1,:]==0):    
               break
    
    list_time_J1[gene]=t
    gene += 1
    
    if t < best_t:
        best_t=t
        best_gantt_chart_J1=gantt_chart_J1
        best_operation_J1=operation_J1

print(list_time_J1)
print(best_gantt_chart_J1)
print(best_operation_J1)        
print('ジョブ1の最適解の作業完成時刻:第',best_t,'時刻')
        
raw_data1=best_gantt_chart_J1
np.savetxt(fname='d:/pysrc/data/gantt_chart_J1.csv',X=raw_data1,fmt='%0d',delimiter=',')
raw_data2=best_operation_J1
np.savetxt(fname='d:/pysrc/data/operation_J1.csv',X=raw_data2,fmt='%0d',delimiter=',')
raw_data3=list_time_J1
np.savetxt(fname='d:/pysrc/data/list_time_J1.csv',X=raw_data3,fmt='%0d',delimiter=',')

[61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61.
 61. 61. 61. 61. 61. 61. 79. 79. 79. 79. 79. 79. 79. 79. 79. 79. 79. 79.
 79. 79. 79. 79. 79. 87. 79. 79. 87. 87. 79. 87. 61. 61. 61. 61. 61. 61.
 79. 79. 79. 79. 79. 79. 61. 61. 79. 79. 61. 79. 61. 61. 79. 79. 61. 79.
 61. 61. 61. 61. 61. 61. 79. 79. 79. 79. 79. 87. 61. 61. 79. 79. 61. 79.
 61. 61. 79. 87. 61. 87. 61. 61. 61. 61. 61. 61. 79. 79. 87. 87. 79. 87.
 61. 61. 87. 87. 61. 87. 61. 61. 79. 87. 61. 87.]
     M1  M2  M3  M4
1     1   0   3   4
2     1   0   3   4
3     1   0   3   4
4     1   0   3   4
5     1   0   3   4
6     1   0   3   4
7     1   0   3   4
8     1   0   3   4
9     2   1   3   4
10    2   1   3   4
11    2   1   3   4
12    2   1   3   4
13    2   1   5   4
14    2   1   5   4
15    2   1   5   3
16    2   1   5   3
17    2   1   5   3
18    2   1   5   3
19    2   1   5   3
20    2   1   5   3
21    2   1   5   3
22    2   1   5   3
23    2   1   0   0
24    2   1   0   0
25    2   4 

In [47]:
#JOBSET2

best_t=9999

MS_J2=np.array([[1,4,-1],
               [2,4,-1],
               [1,3,-1],
               [2,3,4],
               [1,2,4],
               [1,2,3]])

T_J2=np.array([[10,18,-1],
               [10,18,-1],
               [10,20,-1],
               [10,15,12],
               [10,15,12],
               [10,15,12]])

list_arrange_J2 = list(itertools.permutations(np.arange(0,6)))
list_time_J2=np.zeros(math.factorial(6))
gene=0

for a in list_arrange_J2:
    
    gantt_chart_J2=pd.DataFrame({
        'M1':np.zeros(LENGTH),
        'M2':np.zeros(LENGTH),
        'M3':np.zeros(LENGTH),
        'M4':np.zeros(LENGTH)},
        index = [np.arange(1,LENGTH+1)],dtype=int)

    operation_J2=pd.DataFrame({
        'job':np.zeros(WIDE),
        'process':np.zeros(WIDE)},
        index = [np.arange(1,WIDE+1)],dtype=int)
    
    state_M=np.zeros(4,dtype=int)
    state_J2=np.zeros(6,dtype=int)
    check_J2=np.array([[0,0,-1],
                       [0,0,-1],
                       [0,0,-1],
                       [0,0,0],
                       [0,0,0],
                       [0,0,0]]) 
    o=1
    
    for t in np.arange(1,LENGTH+1):
        if np.any(state_M==0) and np.any(state_J2==0):
            for j in a:
                if state_J2[j]==0:
                    for i in np.arange(0,3):
                        if check_J2[j,i]==0 and state_J2[j]==0:
                            if i==0 or (i>0 and check_J2[j,i-1]==1):
                                m=MS_J2[j,i]-1
                                if state_M[m]==0:
                                    state_M[m]=T_J2[j,i]
                                    state_J2[j]=T_J2[j,i]
                                    check_J2[j,i]=1
                                    gantt_chart_J2.iloc[t-1:t-1+T_J2[j,i],m]=j+1
                                    operation_J2.loc[o,'job']=j+1
                                    operation_J2.loc[o,'process']=i+1
                                    o+=1
        state_M[state_M !=0] -= 1
        state_J2[state_J2 !=0] -= 1
        if np.all(check_J2!=0):
            break
   
    for t in np.arange(1,LENGTH+1):
        if np.all(gantt_chart_J2.iloc[t-1,:]==0):    
               break
    
    list_time_J2[gene]=t
    gene += 1
    
    if t < best_t:
        best_t=t
        best_gantt_chart_J2=gantt_chart_J2
        best_operation_J2=operation_J2

print(list_time_J2)
print(best_gantt_chart_J2)
print(best_operation_J2) 
print('ジョブ2の最適解の作業完成時刻:第',best_t,'時刻')        
        
raw_data1=best_gantt_chart_J2
np.savetxt(fname='d:/pysrc/data/gantt_chart_J2.csv',X=raw_data1,fmt='%0d',delimiter=',')
raw_data2=best_operation_J2
np.savetxt(fname='d:/pysrc/data/operation_J2.csv',X=raw_data2,fmt='%0d',delimiter=',')
raw_data3=list_time_J2
np.savetxt(fname='d:/pysrc/data/list_time_J2.csv',X=raw_data3,fmt='%0d',delimiter=',')

[ 73.  80.  73.  73.  80.  80.  73.  73.  71.  73.  71.  71.  71.  71.
  71.  73.  73.  73.  71.  71.  71.  71.  71.  71.  73.  80.  73.  73.
  80.  80.  73.  73.  73.  73.  73.  73.  73.  73.  73.  73.  73.  73.
  80.  80.  73.  73.  80.  73.  73.  73.  71.  73.  71.  71.  73.  73.
  73.  73.  73.  73.  71.  73.  71.  71.  73.  73.  71.  71.  71.  71.
  71.  71.  71.  71.  71.  73.  73.  73.  71.  71.  71.  71.  71.  71.
  71.  73.  71.  71.  73.  73.  73.  73.  73.  73.  73.  73.  71.  71.
  71.  71.  71.  71.  71.  71.  71.  71.  71.  71.  71.  71.  71.  71.
  71.  71.  71.  71.  71.  71.  71.  71.  73.  80.  73.  73.  80.  80.
  73.  73.  71.  73.  71.  71.  71.  71.  71.  73.  73.  73.  71.  71.
  71.  71.  71.  71.  73.  73.  73.  73.  73.  73.  73.  73.  73.  78.
  73.  83.  73.  73.  73.  78.  78.  78.  73.  73.  73.  83.  83.  83.
  73.  73.  71.  73.  71.  71.  73.  73.  73.  78.  73.  83.  71.  73.
  73.  78.  73.  78.  71.  71.  73.  78.  73.  78.  78.  78.  78.  83.
  83. 

In [48]:
#JOBSET3

best_t=9999

MS_J3=np.array([[1,3,-1,-1],
               [2,4,-1,-1],
               [1,2,-1,-1],
               [3,4,-1,-1],
               [1,2,3,4],
               [2,3,4,1]])

T_J3=np.array([[16,15,-1,-1],
               [18,15,-1,-1],
               [20,10,-1,-1],
               [15,10,-1,-1],
               [8,10,15,17],
               [10,15,8,15]])

list_arrange_J3 = list(itertools.permutations(np.arange(0,6)))
list_time_J3=np.zeros(math.factorial(6))
gene=0

for a in list_arrange_J3:
    
    gantt_chart_J3=pd.DataFrame({
        'M1':np.zeros(LENGTH),
        'M2':np.zeros(LENGTH),
        'M3':np.zeros(LENGTH),
        'M4':np.zeros(LENGTH)},
        index = [np.arange(1,LENGTH+1)],dtype=int)

    operation_J3=pd.DataFrame({
        'job':np.zeros(WIDE),
        'process':np.zeros(WIDE)},
        index = [np.arange(1,WIDE+1)],dtype=int)
    
    state_M=np.zeros(4,dtype=int)
    state_J3=np.zeros(6,dtype=int)
    check_J3=np.array([[0,0,-1,-1],
                       [0,0,-1,-1],
                       [0,0,-1,-1],
                       [0,0,-1,-1],
                       [0,0,0,0],
                       [0,0,0,0]]) 
    o=1
    
    for t in np.arange(1,LENGTH+1):
        if np.any(state_M==0) and np.any(state_J3==0):
            for j in a:
                if state_J3[j]==0:
                    for i in np.arange(0,4):
                        if check_J3[j,i]==0 and state_J3[j]==0:
                            if i==0 or (i>0 and check_J3[j,i-1]==1):
                                m=MS_J3[j,i]-1
                                if state_M[m]==0:
                                    state_M[m]=T_J3[j,i]
                                    state_J3[j]=T_J3[j,i]
                                    check_J3[j,i]=1
                                    gantt_chart_J3.iloc[t-1:t-1+T_J3[j,i],m]=j+1
                                    operation_J3.loc[o,'job']=j+1
                                    operation_J3.loc[o,'process']=i+1
                                    o+=1
        state_M[state_M !=0] -= 1
        state_J3[state_J3 !=0] -= 1
        if np.all(check_J3!=0):
            break
   
    for t in np.arange(1,LENGTH+1):
        if np.all(gantt_chart_J3.iloc[t-1,:]==0):    
               break
    
    list_time_J3[gene]=t
    gene += 1
    
    if t < best_t:
        best_t=t
        best_gantt_chart_J3=gantt_chart_J3
        best_operation_J3=operation_J3

print(list_time_J3)
print(best_gantt_chart_J3)
print(best_operation_J3) 
print('ジョブ3の最適解の作業完成時刻:第',best_t,'時刻')        
        
raw_data1=best_gantt_chart_J3
np.savetxt(fname='d:/pysrc/data/gantt_chart_J3.csv',X=raw_data1,fmt='%0d',delimiter=',')
raw_data2=best_operation_J3
np.savetxt(fname='d:/pysrc/data/operation_J3.csv',X=raw_data2,fmt='%0d',delimiter=',')
raw_data3=list_time_J3
np.savetxt(fname='d:/pysrc/data/list_time_J3.csv',X=raw_data3,fmt='%0d',delimiter=',')

[89. 89. 89. 89. 89. 89. 89. 89. 79. 79. 89. 79. 79. 79. 79. 79. 79. 79.
 89. 89. 89. 79. 79. 79. 89. 89. 89. 89. 89. 89. 89. 89. 89. 89. 89. 89.
 89. 89. 89. 89. 89. 89. 89. 89. 89. 89. 89. 89. 89. 89. 79. 79. 89. 79.
 89. 89. 89. 89. 89. 89. 79. 79. 79. 78. 78. 78. 89. 78. 89. 89. 78. 78.
 79. 79. 79. 79. 79. 79. 79. 79. 79. 78. 78. 78. 79. 79. 79. 78. 78. 78.
 78. 78. 78. 78. 78. 78. 89. 89. 89. 78. 78. 78. 89. 89. 89. 89. 89. 89.
 89. 78. 89. 89. 78. 78. 78. 78. 78. 78. 78. 78. 89. 89. 89. 89. 89. 89.
 89. 89. 79. 79. 89. 79. 79. 79. 79. 79. 79. 79. 89. 89. 89. 79. 79. 79.
 91. 91. 91. 91. 91. 91. 91. 91. 92. 92. 91. 92. 92. 92. 92. 92. 92. 92.
 91. 91. 91. 92. 92. 92. 89. 89. 79. 79. 89. 79. 91. 91. 92. 92. 91. 92.
 95. 95. 98. 98. 95. 84. 89. 79. 91. 92. 87. 76. 95. 95. 95. 95. 95. 95.
 98. 98. 98. 98. 98. 98. 95. 95. 98. 98. 95. 84. 95. 95. 84. 84. 95. 84.
 89. 89. 89. 79. 79. 79. 91. 91. 91. 92. 92. 92. 89. 79. 91. 92. 87. 76.
 87. 87. 76. 76. 87. 76. 91. 91. 91. 91. 91. 91. 91

In [49]:
#JOBSET4

best_t=9999

MS_J4=np.array([[4,1,2,-1,-1],
               [3,2,4,-1,-1],
               [2,3,1,3,-1],
               [2,4,1,2,-1],
               [1,2,4,2,3]])

T_J4=np.array([[11,10,7,-1,-1],
               [12,10,8,-1,-1],
               [7,10,9,8,-1],
               [7,8,12,6,-1],
               [9,7,8,10,8]])

list_arrange_J4 = list(itertools.permutations(np.arange(0,5)))
list_time_J4=np.zeros(math.factorial(5))
gene=0

for a in list_arrange_J4:
    gantt_chart_J4 = pd.DataFrame({
        'M1':np.zeros(LENGTH),
        'M2':np.zeros(LENGTH),
        'M3':np.zeros(LENGTH),
        'M4':np.zeros(LENGTH)},
        index = [np.arange(1,LENGTH+1)],dtype=int)
    
    operation_J4=pd.DataFrame({
        'job':np.zeros(WIDE),
        'process':np.zeros(WIDE)},
        index = [np.arange(1,WIDE+1)],dtype=int)
    
    state_M=np.zeros(4,dtype=int)
    state_J4=np.zeros(5,dtype=int)
    check_J4=np.array([[0,0,0,-1,-1],
                       [0,0,0,-1,-1],
                       [0,0,0,0,-1],
                       [0,0,0,0,-1],
                       [0,0,0,0,0]]) 
    o=1
    
    for t in np.arange(1,LENGTH+1):
        if np.any(state_M==0) and np.any(state_J4==0):
            for j in a:
                if state_J4[j]==0:
                    for i in np.arange(0,5):
                        if check_J4[j,i]==0 and state_J4[j]==0:
                            if i==0 or (i>0 and check_J4[j,i-1]==1):
                                m=MS_J4[j,i]-1
                                if state_M[m]==0:
                                    state_M[m]=T_J4[j,i]
                                    state_J4[j]=T_J4[j,i]
                                    check_J4[j,i]=1
                                    gantt_chart_J4.iloc[t-1:t-1+T_J4[j,i],m]=j+1
                                    operation_J4.loc[o,'job']=j+1
                                    operation_J4.loc[o,'process']=i+1
                                    o+=1
        state_M[state_M !=0] -= 1
        state_J4[state_J4 !=0] -= 1
        if np.all(check_J4!=0):
            break

    for t in np.arange(1,LENGTH+1):
        if np.all(gantt_chart_J4.iloc[t-1,:]==0):    
               break
    
    list_time_J4[gene]=t
    gene += 1
    
    if t < best_t:
        best_t=t
        best_gantt_chart_J4=gantt_chart_J4
        best_operation_J4=operation_J4

print(list_time_J4)
print(best_gantt_chart_J4)
print(best_operation_J4)        
print('ジョブ4の最適解の作業完成時刻:第',best_t,'時刻')
        
raw_data1=best_gantt_chart_J4
np.savetxt(fname='d:/pysrc/data/gantt_chart_J4.csv',X=raw_data1,fmt='%0d',delimiter=',')
raw_data2=best_operation_J4
np.savetxt(fname='d:/pysrc/data/operation_J4.csv',X=raw_data2,fmt='%0d',delimiter=',')
raw_data3=list_time_J4
np.savetxt(fname='d:/pysrc/data/list_time_J4.csv',X=raw_data3,fmt='%0d',delimiter=',')

[68. 68. 65. 65. 68. 65. 68. 68. 68. 57. 57. 57. 65. 65. 65. 63. 63. 63.
 57. 59. 57. 57. 59. 59. 68. 68. 65. 65. 68. 65. 68. 68. 68. 59. 59. 59.
 65. 65. 65. 63. 63. 63. 59. 63. 59. 59. 63. 63. 68. 68. 68. 57. 57. 57.
 68. 68. 68. 59. 59. 59. 68. 57. 68. 59. 57. 55. 57. 57. 55. 55. 57. 55.
 65. 65. 65. 63. 63. 63. 65. 65. 65. 63. 63. 63. 65. 63. 65. 63. 63. 58.
 63. 63. 58. 58. 63. 58. 57. 59. 57. 57. 59. 59. 55. 58. 55. 55. 58. 58.
 57. 57. 55. 55. 57. 55. 59. 59. 58. 58. 59. 58.]
     M1  M2  M3  M4
1     5   3   2   1
2     5   3   2   1
3     5   3   2   1
4     5   3   2   1
5     5   3   2   1
6     5   3   2   1
7     5   3   2   1
8     5   4   2   1
9     5   4   2   1
10    0   4   2   1
11    0   4   2   1
12    1   4   2   0
13    1   4   3   0
14    1   4   3   0
15    1   5   3   4
16    1   5   3   4
17    1   5   3   4
18    1   5   3   4
19    1   5   3   4
20    1   5   3   4
21    1   5   3   4
22    0   2   3   4
23    3   2   0   5
24    3   2   0   5
25    3   2 

In [52]:
#JOBSET5

best_t=9999

MS_J5=np.array([[1,2,4],
               [1,3,2],
               [3,4,1],
               [4,2,-1],
               [3,1,-1]])

T_J5=np.array([[6,12,9],
               [18,6,15],
               [9,3,12],
               [6,15,-1],
               [3,9,-1]])

list_arrange_J5 = list(itertools.permutations(np.arange(0,5)))
list_time_J5=np.zeros(math.factorial(5))
gene=0

for a in list_arrange_J5:
    gantt_chart_J5 = pd.DataFrame({
        'M1':np.zeros(LENGTH),
        'M2':np.zeros(LENGTH),
        'M3':np.zeros(LENGTH),
        'M4':np.zeros(LENGTH)},
        index = [np.arange(1,LENGTH+1)],dtype=int)
    
    operation_J5=pd.DataFrame({
        'job':np.zeros(WIDE),
        'process':np.zeros(WIDE)},
        index = [np.arange(1,WIDE+1)],dtype=int)
    
    state_M=np.zeros(4,dtype=int)
    state_J5=np.zeros(5,dtype=int)
    check_J5=np.array([[0,0,0],
                       [0,0,0],
                       [0,0,0],
                       [0,0,-1],
                       [0,0,-1]]) 
    o=1
    
    for t in np.arange(1,LENGTH+1):
        if np.any(state_M==0) and np.any(state_J5==0):
            for j in a:
                if state_J5[j]==0:
                    for i in np.arange(0,3):
                        if check_J5[j,i]==0 and state_J5[j]==0:
                            if i==0 or (i>0 and check_J5[j,i-1]==1):
                                m=MS_J5[j,i]-1
                                if state_M[m]==0:
                                    state_M[m]=T_J5[j,i]
                                    state_J5[j]=T_J5[j,i]
                                    check_J5[j,i]=1
                                    gantt_chart_J5.iloc[t-1:t-1+T_J5[j,i],m]=j+1
                                    operation_J5.loc[o,'job']=j+1
                                    operation_J5.loc[o,'process']=i+1
                                    o+=1
        state_M[state_M !=0] -= 1
        state_J5[state_J5 !=0] -= 1
        if np.all(check_J5!=0):
            break

    for t in np.arange(1,LENGTH+1):
        if np.all(gantt_chart_J5.iloc[t-1,:]==0):    
               break
    
    list_time_J5[gene]=t
    gene += 1
    
    if t < best_t:
        best_t=t
        best_gantt_chart_J5=gantt_chart_J5
        best_operation_J5=operation_J5

print(list_time_J5)
print(best_gantt_chart_J5)
print(best_operation_J5)        
print('ジョブ5の最適解の作業完成時刻:第',best_t,'時刻')
        
raw_data1=best_gantt_chart_J5
np.savetxt(fname='d:/pysrc/data/gantt_chart_J5.csv',X=raw_data1,fmt='%0d',delimiter=',')
raw_data2=best_operation_J5
np.savetxt(fname='d:/pysrc/data/operation_J5.csv',X=raw_data2,fmt='%0d',delimiter=',')
raw_data3=list_time_J5
np.savetxt(fname='d:/pysrc/data/list_time_J5.csv',X=raw_data3,fmt='%0d',delimiter=',')

[49. 49. 49. 49. 49. 49. 49. 49. 49. 49. 49. 49. 49. 49. 49. 49. 55. 67.
 55. 55. 67. 67. 55. 67. 61. 61. 61. 61. 61. 61. 61. 61. 61. 67. 67. 67.
 61. 61. 61. 67. 61. 67. 61. 61. 67. 67. 61. 67. 49. 49. 49. 49. 49. 49.
 61. 61. 61. 67. 67. 67. 49. 49. 61. 67. 49. 67. 49. 49. 67. 67. 49. 67.
 49. 49. 49. 49. 55. 67. 61. 61. 61. 67. 61. 67. 49. 49. 61. 67. 49. 67.
 55. 67. 61. 67. 67. 67. 55. 55. 67. 67. 55. 67. 61. 61. 67. 67. 61. 67.
 67. 67. 67. 67. 67. 67. 55. 67. 61. 67. 67. 67.]
     M1  M2  M3  M4
1     1   0   3   4
2     1   0   3   4
3     1   0   3   4
4     1   0   3   4
5     1   0   3   4
6     1   0   3   4
7     2   1   3   0
8     2   1   3   0
9     2   1   3   0
10    2   1   5   3
11    2   1   5   3
12    2   1   5   3
13    2   1   0   0
14    2   1   0   0
15    2   1   0   0
16    2   1   0   0
17    2   1   0   0
18    2   1   0   0
19    2   4   0   1
20    2   4   0   1
21    2   4   0   1
22    2   4   0   1
23    2   4   0   1
24    2   4   0   1
25    3   4 

In [54]:
#JOBSET6
LENGTH_J6=200
best_t=9999

MS_J6=np.array([[1,2,4],
               [1,2,4],
               [2,3,4],
               [2,3,4],
               [1,3,4],
               [1,3,4]])

T_J6=np.array([[9,11,7],
               [19,20,13],
               [14,20,9],
               [14,20,9],
               [11,16,8],
               [10,12,10]])

list_arrange_J6 = list(itertools.permutations(np.arange(0,6)))
list_time_J6=np.zeros(math.factorial(6))
gene=0

for a in list_arrange_J6:
    
    gantt_chart_J6=pd.DataFrame({
        'M1':np.zeros(LENGTH_J6),
        'M2':np.zeros(LENGTH_J6),
        'M3':np.zeros(LENGTH_J6),
        'M4':np.zeros(LENGTH_J6)},
        index = [np.arange(1,LENGTH_J6+1)],dtype=int)

    operation_J6=pd.DataFrame({
        'job':np.zeros(WIDE),
        'process':np.zeros(WIDE)},
        index = [np.arange(1,WIDE+1)],dtype=int)
    
    state_M=np.zeros(4,dtype=int)
    state_J6=np.zeros(6,dtype=int)
    check_J6=np.array([[0,0,0],
                       [0,0,0],
                       [0,0,0],
                       [0,0,0],
                       [0,0,0],
                       [0,0,0]]) 
    o=1
    
    for t in np.arange(1,LENGTH_J6+1):
        if np.any(state_M==0) and np.any(state_J6==0):
            for j in a:
                if state_J6[j]==0:
                    for i in np.arange(0,3):
                        if check_J6[j,i]==0 and state_J6[j]==0:
                            if i==0 or (i>0 and check_J6[j,i-1]==1):
                                m=MS_J6[j,i]-1
                                if state_M[m]==0:
                                    state_M[m]=T_J6[j,i]
                                    state_J6[j]=T_J6[j,i]
                                    check_J6[j,i]=1
                                    gantt_chart_J6.iloc[t-1:t-1+T_J6[j,i],m]=j+1
                                    operation_J6.loc[o,'job']=j+1
                                    operation_J6.loc[o,'process']=i+1
                                    o+=1
        state_M[state_M !=0] -= 1
        state_J6[state_J6 !=0] -= 1
        if np.all(check_J6!=0):
            break
   
    for t in np.arange(1,LENGTH_J6+1):
        if np.all(gantt_chart_J6.iloc[t-1,:]==0):    
               break
    
    list_time_J6[gene]=t
    gene += 1
    
    if t < best_t:
        best_t=t
        best_gantt_chart_J6=gantt_chart_J6
        best_operation_J6=operation_J6

print(list_time_J6)
print(best_gantt_chart_J6)
print(best_operation_J6) 
print('ジョブ6の最適解の作業完成時刻:第',best_t,'時刻')        
        
raw_data1=best_gantt_chart_J6
np.savetxt(fname='d:/pysrc/data/gantt_chart_J6.csv',X=raw_data1,fmt='%0d',delimiter=',')
raw_data2=best_operation_J6
np.savetxt(fname='d:/pysrc/data/operation_J6.csv',X=raw_data2,fmt='%0d',delimiter=',')
raw_data3=list_time_J6
np.savetxt(fname='d:/pysrc/data/list_time_J6.csv',X=raw_data3,fmt='%0d',delimiter=',')

[100.  95.  98.  97.  95.  96. 100.  95.  98.  97.  95.  96.  98.  97.
  98.  97.  97.  97.  95.  96.  95.  96.  96.  96. 100.  95.  98.  97.
  95.  96. 100.  95.  93. 102.  91.  97.  93.  92.  93. 102.  95.  95.
  91.  91.  91.  97.  93.  93. 100.  95.  98.  97.  95.  96. 100.  95.
  93. 102.  91.  97.  93.  92.  93. 102.  95.  95.  91.  91.  91.  97.
  93.  93.  93.  92.  93.  92.  92.  92.  93.  92.  93. 102.  95.  95.
  93.  92.  93. 102.  95.  95.  95.  95.  95.  95.  95.  95.  91.  91.
  91.  91.  91.  91.  91.  91.  91.  97.  93.  93.  91.  91.  91.  97.
  93.  93.  93.  93.  93.  93.  93.  93.  96.  96.  96.  96.  96.  96.
  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.
  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.
  96.  96.  96.  96.  96.  96.  96.  96.  94.  94.  94.  94.  94.  94.
  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.  96.
  96.  96.  96.  96.  94.  94.  94.  94.  94.  94.  96.  96.  96.  96.
  96. 

In [56]:
#JOBSET7
best_t=9999

MS_J7=np.array([[1,4,-1],
               [2,4,-1],
               [2,4,-1],
               [3,4,-1],
               [1,3,-1],
               [2,3,4],
               [1,2,3],
               [1,2,4]])

T_J7=np.array([[6,6,-1],
               [11,9,-1],
               [9,7,-1],
               [16,7,-1],
               [9,18,-1],
               [13,19,6],
               [10,9,13],
               [11,9,8]])

list_arrange_J7 = list(itertools.permutations(np.arange(0,8)))
list_time_J7=np.zeros(math.factorial(8))
gene=0

for a in list_arrange_J7:
    
    gantt_chart_J7=pd.DataFrame({
        'M1':np.zeros(LENGTH),
        'M2':np.zeros(LENGTH),
        'M3':np.zeros(LENGTH),
        'M4':np.zeros(LENGTH)},
        index = [np.arange(1,LENGTH+1)],dtype=int)

    operation_J7=pd.DataFrame({
        'job':np.zeros(WIDE),
        'process':np.zeros(WIDE)},
        index = [np.arange(1,WIDE+1)],dtype=int)
    
    state_M=np.zeros(4,dtype=int)
    state_J7=np.zeros(8,dtype=int)
    check_J7=np.array([[0,0,-1],
                       [0,0,-1],
                       [0,0,-1],
                       [0,0,-1],
                       [0,0,-1],
                       [0,0,0],
                       [0,0,0],
                       [0,0,0],]) 
    o=1
    
    for t in np.arange(1,LENGTH+1):
        if np.any(state_M==0) and np.any(state_J7==0):
            for j in a:
                if state_J7[j]==0:
                    for i in np.arange(0,3):
                        if check_J7[j,i]==0 and state_J7[j]==0:
                            if i==0 or (i>0 and check_J7[j,i-1]==1):
                                m=MS_J7[j,i]-1
                                if state_M[m]==0:
                                    state_M[m]=T_J7[j,i]
                                    state_J7[j]=T_J7[j,i]
                                    check_J7[j,i]=1
                                    gantt_chart_J7.iloc[t-1:t-1+T_J7[j,i],m]=j+1
                                    operation_J7.loc[o,'job']=j+1
                                    operation_J7.loc[o,'process']=i+1
                                    o+=1
        state_M[state_M !=0] -= 1
        state_J7[state_J7 !=0] -= 1
        if np.all(check_J7!=0):
            break
   
    for t in np.arange(1,LENGTH+1):
        if np.all(gantt_chart_J7.iloc[t-1,:]==0):    
               break
    
    list_time_J7[gene]=t
    gene += 1
    
    if t < best_t:
        best_t=t
        best_gantt_chart_J7=gantt_chart_J7
        best_operation_J7=operation_J7

print(list_time_J7)
print(best_gantt_chart_J7)
print(best_operation_J7) 
print('ジョブ7の最適解の作業完成時刻:第',best_t,'時刻')        
        
raw_data1=best_gantt_chart_J7
np.savetxt(fname='d:/pysrc/data/gantt_chart_J7.csv',X=raw_data1,fmt='%0d',delimiter=',')
raw_data2=best_operation_J7
np.savetxt(fname='d:/pysrc/data/operation_J7.csv',X=raw_data2,fmt='%0d',delimiter=',')
raw_data3=list_time_J7
np.savetxt(fname='d:/pysrc/data/list_time_J7.csv',X=raw_data3,fmt='%0d',delimiter=',')

[67. 67. 67. ... 67. 67. 67.]
     M1  M2  M3  M4
1     1   2   4   0
2     1   2   4   0
3     1   2   4   0
4     1   2   4   0
5     1   2   4   0
6     1   2   4   0
7     5   2   4   1
8     5   2   4   1
9     5   2   4   1
10    5   2   4   1
11    5   2   4   1
12    5   3   4   1
13    5   3   4   2
14    5   3   4   2
15    5   3   4   2
16    7   3   4   2
17    7   3   5   2
18    7   3   5   2
19    7   3   5   2
20    7   3   5   2
21    7   6   5   2
22    7   6   5   3
23    7   6   5   3
24    7   6   5   3
25    7   6   5   3
26    8   6   5   3
27    8   6   5   3
28    8   6   5   3
29    8   6   5   4
30    8   6   5   4
31    8   6   5   4
32    8   6   5   4
33    8   6   5   4
34    8   7   5   4
35    8   7   6   4
36    8   7   6   0
37    0   7   6   0
38    0   7   6   0
39    0   7   6   0
40    0   7   6   0
41    0   7   6   0
42    0   7   6   0
43    0   8   6   0
44    0   8   6   0
45    0   8   6   0
46    0   8   6   0
47    0   8   6   0
48    0   

In [59]:
#JOBSET8

best_t=9999
LENGTH_J8=200

MS_J8=np.array([[2,3,4,-1],
               [2,3,4,-1],
               [2,3,4,-1],
               [2,3,4,-1],
               [1,2,3,4],
               [1,2,3,4]])

T_J8=np.array([[12,21,11,-1],
               [12,21,11,-1],
               [12,21,11,-1],
               [12,21,11,-1],
               [10,14,18,9],
               [10,14,18,9]])

list_arrange_J8 = list(itertools.permutations(np.arange(0,6)))
list_time_J8=np.zeros(math.factorial(6))
gene=0

for a in list_arrange_J8:
    
    gantt_chart_J8=pd.DataFrame({
        'M1':np.zeros(LENGTH_J8),
        'M2':np.zeros(LENGTH_J8),
        'M3':np.zeros(LENGTH_J8),
        'M4':np.zeros(LENGTH_J8)},
        index = [np.arange(1,LENGTH_J8+1)],dtype=int)

    operation_J8=pd.DataFrame({
        'job':np.zeros(WIDE),
        'process':np.zeros(WIDE)},
        index = [np.arange(1,WIDE+1)],dtype=int)
    
    state_M=np.zeros(4,dtype=int)
    state_J8=np.zeros(6,dtype=int)
    check_J8=np.array([[0,0,0,-1],
                       [0,0,0,-1],
                       [0,0,0,-1],
                       [0,0,0,-1],
                       [0,0,0,0],
                       [0,0,0,0]]) 
    o=1
    
    for t in np.arange(1,LENGTH_J8+1):
        if np.any(state_M==0) and np.any(state_J8==0):
            for j in a:
                if state_J8[j]==0:
                    for i in np.arange(0,4):
                        if check_J8[j,i]==0 and state_J8[j]==0:
                            if i==0 or (i>0 and check_J8[j,i-1]==1):
                                m=MS_J8[j,i]-1
                                if state_M[m]==0:
                                    state_M[m]=T_J8[j,i]
                                    state_J8[j]=T_J8[j,i]
                                    check_J8[j,i]=1
                                    gantt_chart_J8.iloc[t-1:t-1+T_J8[j,i],m]=j+1
                                    operation_J8.loc[o,'job']=j+1
                                    operation_J8.loc[o,'process']=i+1
                                    o+=1
        state_M[state_M !=0] -= 1
        state_J8[state_J8 !=0] -= 1
        if np.all(check_J8!=0):
            break
   
    for t in np.arange(1,LENGTH_J8+1):
        if np.all(gantt_chart_J8.iloc[t-1,:]==0):    
               break
    
    list_time_J8[gene]=t
    gene += 1
    
    if t < best_t:
        best_t=t
        best_gantt_chart_J8=gantt_chart_J8
        best_operation_J8=operation_J8

print(list_time_J8)
print(best_gantt_chart_J8)
print(best_operation_J8) 
print('ジョブ8の最適解の作業完成時刻:第',best_t,'時刻')        
        
raw_data1=best_gantt_chart_J8
np.savetxt(fname='d:/pysrc/data/gantt_chart_J8.csv',X=raw_data1,fmt='%0d',delimiter=',')
raw_data2=best_operation_J8
np.savetxt(fname='d:/pysrc/data/operation_J8.csv',X=raw_data2,fmt='%0d',delimiter=',')
raw_data3=list_time_J8
np.savetxt(fname='d:/pysrc/data/list_time_J8.csv',X=raw_data3,fmt='%0d',delimiter=',')

[142. 142. 142. 144. 142. 144. 142. 142. 142. 144. 142. 144. 142. 144.
 142. 144. 144. 144. 142. 144. 142. 144. 144. 144. 142. 142. 142. 144.
 142. 144. 142. 142. 142. 144. 142. 144. 142. 144. 142. 144. 144. 144.
 142. 144. 142. 144. 144. 144. 142. 142. 142. 144. 142. 144. 142. 142.
 142. 144. 142. 144. 142. 144. 142. 144. 144. 144. 142. 144. 142. 144.
 144. 144. 142. 144. 142. 144. 144. 144. 142. 144. 142. 144. 144. 144.
 142. 144. 142. 144. 144. 144. 144. 144. 144. 144. 144. 144. 142. 144.
 142. 144. 144. 144. 142. 144. 142. 144. 144. 144. 142. 144. 142. 144.
 144. 144. 144. 144. 144. 144. 144. 144. 142. 142. 142. 144. 142. 144.
 142. 142. 142. 144. 142. 144. 142. 144. 142. 144. 144. 144. 142. 144.
 142. 144. 144. 144. 142. 142. 142. 144. 142. 144. 142. 142. 142. 144.
 142. 144. 142. 144. 142. 144. 144. 144. 142. 144. 142. 144. 144. 144.
 142. 142. 142. 144. 142. 144. 142. 142. 142. 144. 142. 144. 142. 144.
 142. 144. 144. 144. 142. 144. 142. 144. 144. 144. 142. 144. 142. 144.
 144. 

In [61]:
#JOBSET9

best_t=9999
LENGTH_J9=200

MS_J9=np.array([[3,1,2,4],
               [3,2,4,-1],
               [1,2,4,-1],
               [2,3,4,-1],
               [3,1,2,4]])

T_J9=np.array([[9,12,9,6],
               [16,11,9,-1],
               [21,18,7,-1],
               [20,22,11,-1],
               [14,16,13,9]])

list_arrange_J9 = list(itertools.permutations(np.arange(0,5)))
list_time_J9=np.zeros(math.factorial(5))
gene=0

for a in list_arrange_J9:
    gantt_chart_J9 = pd.DataFrame({
        'M1':np.zeros(LENGTH_J9),
        'M2':np.zeros(LENGTH_J9),
        'M3':np.zeros(LENGTH_J9),
        'M4':np.zeros(LENGTH_J9)},
        index = [np.arange(1,LENGTH_J9+1)],dtype=int)
    
    operation_J9=pd.DataFrame({
        'job':np.zeros(WIDE),
        'process':np.zeros(WIDE)},
        index = [np.arange(1,WIDE+1)],dtype=int)
    
    state_M=np.zeros(4,dtype=int)
    state_J9=np.zeros(5,dtype=int)
    check_J9=np.array([[0,0,0,0],
                       [0,0,0,-1],
                       [0,0,0,-1],
                       [0,0,0,-1],
                       [0,0,0,0]]) 
    o=1
    
    for t in np.arange(1,LENGTH_J9+1):
        if np.any(state_M==0) and np.any(state_J9==0):
            for j in a:
                if state_J9[j]==0:
                    for i in np.arange(0,4):
                        if check_J9[j,i]==0 and state_J9[j]==0:
                            if i==0 or (i>0 and check_J9[j,i-1]==1):
                                m=MS_J9[j,i]-1
                                if state_M[m]==0:
                                    state_M[m]=T_J9[j,i]
                                    state_J9[j]=T_J9[j,i]
                                    check_J9[j,i]=1
                                    gantt_chart_J9.iloc[t-1:t-1+T_J9[j,i],m]=j+1
                                    operation_J9.loc[o,'job']=j+1
                                    operation_J9.loc[o,'process']=i+1
                                    o+=1
        state_M[state_M !=0] -= 1
        state_J9[state_J9 !=0] -= 1
        if np.all(check_J9!=0):
            break

    for t in np.arange(1,LENGTH_J9+1):
        if np.all(gantt_chart_J9.iloc[t-1,:]==0):    
               break
    
    list_time_J9[gene]=t
    gene += 1
    
    if t < best_t:
        best_t=t
        best_gantt_chart_J9=gantt_chart_J9
        best_operation_J9=operation_J9

print(list_time_J9)
print(best_gantt_chart_J9)
print(best_operation_J9)        
print('ジョブ9の最適解の作業完成時刻:第',best_t,'時刻')
        
raw_data1=best_gantt_chart_J9
np.savetxt(fname='d:/pysrc/data/gantt_chart_J9.csv',X=raw_data1,fmt='%0d',delimiter=',')
raw_data2=best_operation_J9
np.savetxt(fname='d:/pysrc/data/operation_J9.csv',X=raw_data2,fmt='%0d',delimiter=',')
raw_data3=list_time_J9
np.savetxt(fname='d:/pysrc/data/list_time_J9.csv',X=raw_data3,fmt='%0d',delimiter=',')

[100.  89. 100. 100.  89.  89. 100.  89. 100.  83.  89.  83. 100. 100.
 100.  83.  83.  83.  89.  89.  89.  83.  83.  83. 100.  85. 100. 100.
  85.  85. 100.  85. 100.  89.  88.  89. 100. 100. 100.  89.  89.  89.
  88.  88.  88.  89.  89.  89. 100.  89. 100.  83.  89.  83. 100.  85.
 100.  89.  88.  89. 100.  83. 100.  89.  82.  89.  88.  82.  88.  89.
  82.  89. 100. 100. 100.  83.  83.  83. 100. 100. 100.  89.  89.  89.
 100.  83. 100.  89.  82.  89.  82.  82.  89.  89.  82.  89.  88.  88.
  88.  82.  82.  82.  88.  88.  88.  89.  89.  89.  88.  82.  88.  89.
  82.  89.  82.  82.  89.  89.  82.  89.]
     M1  M2  M3  M4
1     3   4   5   0
2     3   4   5   0
3     3   4   5   0
4     3   4   5   0
5     3   4   5   0
6     3   4   5   0
7     3   4   5   0
8     3   4   5   0
9     3   4   5   0
10    3   4   5   0
11    3   4   5   0
12    3   4   5   0
13    3   4   5   0
14    3   4   5   0
15    3   4   1   0
16    3   4   1   0
17    3   4   1   0
18    3   4   1   0
19    3   

In [62]:
#JOBSET10

best_t=9999
LENGTH_J10=200

MS_J10=np.array([[1,3,2,4],
               [2,3,4,-1],
               [3,2,1,4],
               [2,3,4,-1],
               [1,3,4,-1],
               [2,1,3,4]])

T_J10=np.array([[11,19,16,13],
               [21,16,14,-1],
               [8,10,14,9],
               [13,20,10,-1],
               [9,16,18,-1],
               [19,21,11,15]])

list_arrange_J10 = list(itertools.permutations(np.arange(0,6)))
list_time_J10=np.zeros(math.factorial(6))
gene=0

for a in list_arrange_J10:
    
    gantt_chart_J10=pd.DataFrame({
        'M1':np.zeros(LENGTH_J10),
        'M2':np.zeros(LENGTH_J10),
        'M3':np.zeros(LENGTH_J10),
        'M4':np.zeros(LENGTH_J10)},
        index = [np.arange(1,LENGTH_J10+1)],dtype=int)

    operation_J10=pd.DataFrame({
        'job':np.zeros(WIDE),
        'process':np.zeros(WIDE)},
        index = [np.arange(1,WIDE+1)],dtype=int)
    
    state_M=np.zeros(4,dtype=int)
    state_J10=np.zeros(6,dtype=int)
    check_J10=np.array([[0,0,0,0],
                       [0,0,0,-1],
                       [0,0,0,0],
                       [0,0,0,-1],
                       [0,0,0,-1],
                       [0,0,0,0]]) 
    o=1
    
    for t in np.arange(1,LENGTH_J10+1):
        if np.any(state_M==0) and np.any(state_J10==0):
            for j in a:
                if state_J10[j]==0:
                    for i in np.arange(0,4):
                        if check_J10[j,i]==0 and state_J10[j]==0:
                            if i==0 or (i>0 and check_J10[j,i-1]==1):
                                m=MS_J10[j,i]-1
                                if state_M[m]==0:
                                    state_M[m]=T_J10[j,i]
                                    state_J10[j]=T_J10[j,i]
                                    check_J10[j,i]=1
                                    gantt_chart_J10.iloc[t-1:t-1+T_J10[j,i],m]=j+1
                                    operation_J10.loc[o,'job']=j+1
                                    operation_J10.loc[o,'process']=i+1
                                    o+=1
        state_M[state_M !=0] -= 1
        state_J10[state_J10 !=0] -= 1
        if np.all(check_J10!=0):
            break
   
    for t in np.arange(1,LENGTH_J10+1):
        if np.all(gantt_chart_J10.iloc[t-1,:]==0):    
               break
    
    list_time_J10[gene]=t
    gene += 1
    
    if t < best_t:
        best_t=t
        best_gantt_chart_J10=gantt_chart_J10
        best_operation_J10=operation_J10

print(list_time_J10)
print(best_gantt_chart_J10)
print(best_operation_J10) 
print('ジョブ10の最適解の作業完成時刻:第',best_t,'時刻')        
    
raw_data1=best_gantt_chart_J10
np.savetxt(fname='d:/pysrc/data/gantt_chart_J10.csv',X=raw_data1,fmt='%0d',delimiter=',')
raw_data2=best_operation_J10
np.savetxt(fname='d:/pysrc/data/operation_J10.csv',X=raw_data2,fmt='%0d',delimiter=',')
raw_data3=list_time_J10
np.savetxt(fname='d:/pysrc/data/list_time_J10.csv',X=raw_data3,fmt='%0d',delimiter=',')

[127. 127. 127. 126. 126. 126. 127. 127. 127. 126. 126. 126. 127. 126.
 127. 126. 126. 126. 126. 126. 126. 126. 126. 126. 127. 127. 127. 126.
 126. 126. 127. 127. 127. 121. 121. 121. 127. 126. 127. 117. 126. 126.
 126. 126. 126. 126. 126. 126. 130. 130. 130. 130. 130. 130. 127. 127.
 127. 121. 121. 121. 130. 130. 127. 121. 128. 128. 128. 128. 128. 128.
 128. 128. 127. 126. 127. 126. 126. 126. 127. 126. 127. 117. 126. 126.
 127. 126. 127. 117. 126. 126. 126. 126. 126. 126. 126. 126. 126. 126.
 126. 126. 126. 126. 126. 126. 126. 126. 126. 126. 126. 126. 126. 126.
 126. 126. 126. 126. 126. 126. 126. 126. 127. 127. 127. 126. 126. 126.
 127. 127. 127. 126. 126. 126. 127. 126. 127. 126. 126. 126. 126. 126.
 126. 126. 126. 126. 127. 127. 127. 126. 126. 126. 127. 127. 118. 118.
 126. 118. 118. 118. 118. 118. 118. 118. 125. 125. 125. 118. 118. 118.
 127. 127. 127. 126. 126. 126. 127. 127. 124. 124. 126. 124. 124. 126.
 124. 124. 126. 126. 132. 132. 135. 126. 126. 126. 118. 118. 118. 127.
 120. 